In [1]:
from datetime import datetime, timedelta, date
from subgrounds.subgrounds import Subgrounds
import pandas as pd
import os

C:\Users\ianfe\PycharmProjects\pythonProject\venv\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
sg = Subgrounds()
holders = sg.load_api('https://api.studio.thegraph.com/query/28103/token-holders/0.0.40')

In [3]:
    start_date = datetime(2021,11,24)
    today = datetime.today()
    delta = timedelta(hours=1)

In [4]:
def do_query(holders, page, start_date: datetime, finish_date: datetime, df):
    # Fetch the next page and concatenate
    ohm_transactions = holders.Query.tokenHolderTransactions(
        orderBy="timestamp",
        orderDirection="asc",
        first=1000,
        skip=1000*page,
        where=[
            holders.TokenHolderTransaction.date >= start_date.strftime("%Y-%m-%dT%H:%M:%S.000Z"),
            holders.TokenHolderTransaction.date < finish_date.strftime("%Y-%m-%dT%H:%M:%S.000Z"),
        ]
    )
    print("Fetching records for date: ", start_date)
    print("page: ", page)
    events = sg.query_df([
        ohm_transactions.date,
        ohm_transactions.holder,
        ohm_transactions.id,
        ohm_transactions.transaction,
        ohm_transactions.type,
        ohm_transactions.value,
        ohm_transactions.holder.token.name,
        ohm_transactions.holder.token.blockchain,
        ohm_transactions.holder.token.name,
        ohm_transactions.balance,
        ohm_transactions.previousBalance
    ])
    print("records: ", len(events))
    updated_df = pd.concat([df, events])

    if len(events) < 1000:
        return updated_df
    return do_query(holders, page+1, start_date, finish_date, updated_df)

df = pd.DataFrame()

while start_date <= today:
    next_date = start_date + delta

    # Fetch recursively
    df = do_query(holders, 0, start_date, next_date, df)

    # Increment start date for the next loop
    start_date += delta

# Cache the results
filename = "results"+str(date.today().strftime("%b-%d-%Y"))+".csv"
df.to_csv(filename, float_format="%.18f")
print("Wrote cached results")



Fetching records for date:  2021-11-24 00:00:00
page:  0
records:  0
Fetching records for date:  2021-11-24 01:00:00
page:  0
records:  0
Fetching records for date:  2021-11-24 02:00:00
page:  0
records:  0
Fetching records for date:  2021-11-24 03:00:00
page:  0
records:  0
Fetching records for date:  2021-11-24 04:00:00
page:  0
records:  0
Fetching records for date:  2021-11-24 05:00:00
page:  0
records:  0
Fetching records for date:  2021-11-24 06:00:00
page:  0
records:  0
Fetching records for date:  2021-11-24 07:00:00
page:  0
records:  0
Fetching records for date:  2021-11-24 08:00:00
page:  0
records:  5
Fetching records for date:  2021-11-24 09:00:00
page:  0
records:  0
Fetching records for date:  2021-11-24 10:00:00
page:  0
records:  0
Fetching records for date:  2021-11-24 11:00:00
page:  0
records:  0
Fetching records for date:  2021-11-24 12:00:00
page:  0
records:  0
Fetching records for date:  2021-11-24 13:00:00
page:  0
records:  0
Fetching records for date:  2021-1

KeyboardInterrupt: 

In [5]:
df.head(10)

,tokenHolderTransactions_date,tokenHolderTransactions_holder_id,tokenHolderTransactions_holder_balance,tokenHolderTransactions_holder_holder,tokenHolderTransactions_holder_token_id,tokenHolderTransactions_id,tokenHolderTransactions_transaction,tokenHolderTransactions_type,tokenHolderTransactions_value,tokenHolderTransactions_holder_token_name,tokenHolderTransactions_holder_token_blockchain,tokenHolderTransactions_balance,tokenHolderTransactions_previousBalance
0,2021-11-24T08:18:17.000Z,gOHM/Ethereum/0xf9704b03e94b8c19cfd8a8803d81c9...,0.000000e+00,0xf9704b03e94b8c19cfd8a8803d81c95e814d2a44,gOHM/Ethereum,gOHM/Ethereum/0xf9704b03e94b8c19cfd8a8803d81c9...,0x1801ffca3dc2effe1dc34111057fe85dbe64c213a852...,TRANSFER,5.358220e-04,gOHM,Ethereum,5.358220e-04,0.000000
1,2021-11-24T08:21:28.000Z,gOHM/Ethereum/0xf9704b03e94b8c19cfd8a8803d81c9...,0.000000e+00,0xf9704b03e94b8c19cfd8a8803d81c95e814d2a44,gOHM/Ethereum,gOHM/Ethereum/0xf9704b03e94b8c19cfd8a8803d81c9...,0x206f8b4a3ddf9bf3ca9b1d5ef6994d6f4d979db51d93...,TRANSFER,-5.358220e-04,gOHM,Ethereum,0.000000e+00,0.000536
2,2021-11-24T08:22:37.000Z,gOHM/Ethereum/0xf9704b03e94b8c19cfd8a8803d81c9...,0.000000e+00,0xf9704b03e94b8c19cfd8a8803d81c95e814d2a44,gOHM/Ethereum,gOHM/Ethereum/0xf9704b03e94b8c19cfd8a8803d81c9...,0x9d22aa742faedd1dc2d63c1c2643b296c6178b52300f...,TRANSFER,5.368538e-04,gOHM,Ethereum,5.368538e-04,0.000000
3,2021-11-24T08:58:07.000Z,gOHM/Ethereum/0x5d8b27b839d86edf67a334139e8d51...,1.000000e-09,0x5d8b27b839d86edf67a334139e8d51a5348eeba2,gOHM/Ethereum,gOHM/Ethereum/0x5d8b27b839d86edf67a334139e8d51...,0x7a5a72edbabd5011dc607c1a967da5d701b598f06ade...,TRANSFER,1.000000e-09,gOHM,Ethereum,1.000000e-09,0.000000
4,2021-11-24T08:58:07.000Z,gOHM/Ethereum/0xf9704b03e94b8c19cfd8a8803d81c9...,0.000000e+00,0xf9704b03e94b8c19cfd8a8803d81c95e814d2a44,gOHM/Ethereum,gOHM/Ethereum/0xf9704b03e94b8c19cfd8a8803d81c9...,0x7a5a72edbabd5011dc607c1a967da5d701b598f06ade...,TRANSFER,-1.000000e-09,gOHM,Ethereum,5.368528e-04,0.000537
0,2021-11-27T02:39:44.000Z,gOHM/Ethereum/0x0af91fa049a7e1894f480bfe5bba20...,0.000000e+00,0x0af91fa049a7e1894f480bfe5bba20142c6c29a9,gOHM/Ethereum,gOHM/Ethereum/0x0af91fa049a7e1894f480bfe5bba20...,0xdb5632348761e5393b92c66873336a4544b07558ea27...,TRANSFER,5.368528e-04,gOHM,Ethereum,5.368528e-04,0.000000
1,2021-11-27T02:39:44.000Z,gOHM/Ethereum/0xf9704b03e94b8c19cfd8a8803d81c9...,0.000000e+00,0xf9704b03e94b8c19cfd8a8803d81c95e814d2a44,gOHM/Ethereum,gOHM/Ethereum/0xf9704b03e94b8c19cfd8a8803d81c9...,0xdb5632348761e5393b92c66873336a4544b07558ea27...,TRANSFER,-5.368528e-04,gOHM,Ethereum,0.000000e+00,0.000537
0,2021-11-27T18:41:18.000Z,gOHM/Ethereum/0x0af91fa049a7e1894f480bfe5bba20...,0.000000e+00,0x0af91fa049a7e1894f480bfe5bba20142c6c29a9,gOHM/Ethereum,gOHM/Ethereum/0x0af91fa049a7e1894f480bfe5bba20...,0xddf8d1147a35cb70e238215640e68bd0c1f770f95ef6...,TRANSFER,-5.368528e-04,gOHM,Ethereum,0.000000e+00,0.000537
1,2021-11-27T18:41:18.000Z,gOHM/Ethereum/0x42980e3e602178354e065723d9652b...,0.000000e+00,0x42980e3e602178354e065723d9652bef79ae3673,gOHM/Ethereum,gOHM/Ethereum/0x42980e3e602178354e065723d9652b...,0xddf8d1147a35cb70e238215640e68bd0c1f770f95ef6...,TRANSFER,5.368528e-04,gOHM,Ethereum,5.368528e-04,0.000000
0,2021-11-27T19:27:54.000Z,gOHM/Ethereum/0x2796317b0ff8538f253012862c0678...,1.178721e+04,0x2796317b0ff8538f253012862c06787adfb8ceb6,gOHM/Ethereum,gOHM/Ethereum/0x2796317b0ff8538f253012862c0678...,0x7ef5f971b4f984d3c54ffc17527f97905b3936b61a92...,TRANSFER,5.368528e-04,gOHM,Ethereum,5.368528e-04,0.000000
